# Flask-SQLAlchemy ORM - Praca z danymi (komendy DML i DQL)

### Co potrzebujemy do uruchomienia tego notebooka?
* jupyter
* Flask-SQLAlchemy

<code>$ pip install jupyter Flask-SQLAlchemy</code>

Więcej informacji znajdziesz [tutaj](https://stackoverflow.com/questions/39773125/use-flask-sqlalchemy-models-in-jupyter-notebook)

In [ ]:
### Połączenie z bazą 

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass

db = SQLAlchemy(model_class=Base)

In [ ]:
# create the app
app = Flask(__name__)

# configure the SQLite database, relative to the app instance folder
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.sqlite3"

# initialize the app with the extension (instance folder is created)
db.init_app(app)

## Definicja modelu

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

In [ ]:
from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column

class Task(db.Model):
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String)

In [ ]:
app.app_context().push()

In [ ]:
db.create_all()

# optionally
# with app.app_context():
    # db.create_all()

## C - CREATE (CRUD) - DML (Data Manipulation Language)

### Klauzula INSERT

In [ ]:
# Metoda I - metoda add sesji

task = Task(name="Sprzątanie")
db.session.add(task)
db.session.commit()

In [ ]:
# Metoda II - metoda add_all sesji

db.session.add_all([
    Task(name="Pisanie"),
    Task(name="Zamiatanie"),
    Task(name="Malowanie")
])
db.session.commit()

In [ ]:
# Przed przejściem do litery R dodajmy jeszcze kilka wpisów do tabeli, żeby mieć 
# co analizować podczas poznawania instrukcji DQL.

db.session.add_all([
    Task(name="Szukanie"),
    Task(name="Szukanie"),
    Task(name="Programowanie"),
    Task(name="Pływanie"),
    Task(name="Pranie"),
    Task(name="Dodawanie"),
])
db.session.commit()

## R - Read (CRUD) - DQL (Data Query Language)

### Klauzula SELECT

Operacja READ w SQL to instrukcja SELECT z całą swoją rozbudowaną składnią i operatorami takimi jak: LIKE, GROUP_BY, ORDER_BY, HAVING, IN, JOIN, UNION, ...
Metoda **.query** sesji posiada odpowiednie metody implementujące część tych instrukcji.

In [ ]:
print(dir(Task))

In [ ]:
# metoda query sesji
print(db.session.query(Task))

In [ ]:
print(type(db.session.query(Task)))

In [ ]:
print(dir(db.session.query(Task)))

In [ ]:
# metoda `all` obiektu `query`
result = db.session.query(Task).all()
print(result)

In [ ]:
print(type(result))

In [ ]:
print(result[0])

In [ ]:
print(type(result[0]))

In [ ]:
for entry in result:
    print(entry)

In [ ]:
for entry in result:
    print(entry.name)

In [ ]:
# jeżeli nie chcemy z bazy wyciągać wszystkiego, tylko wartości w wybranych kolumnach, to kolumny podajemy jako parametry metody query.
result = db.session.query(Task.name)
print(result)

In [ ]:
print(result[0])

In [ ]:
print(type(result[0]))

In [ ]:
print(dir(result[0]))

In [ ]:
print(result[0][0])

In [ ]:
print(result[0].name)

Możemy też użyć metody query modelu.

In [ ]:
print(Task.query)

In [ ]:
print(type(Task.query))

In [ ]:
tasks = Task.query.all()

print(tasks)

Metoda all zwraca listę instancji modelu.

In [ ]:
print(tasks.query)

Wśród wielu metod udostępnianych przez query można znaleźć all, filter, filter_by, exists, union, get, firstFD, order_by, ...

Popatrzmy na wybrane

In [ ]:
tasks = Task.query.order_by('name')

print(tasks)

In [ ]:
print(type(tasks))

Metoda order_by też zwraca obiekt klasy Query na którym możemy dalej działać. Skoro to co zwraca metoda order_by to obiekt klasy Query, to ten obiekt posiada takie metody jak filter_by, order_by, ... Wynika z tego, że te metody możemy łańcuchować, tzn. wywoływać jedna po drugiej.

In [ ]:
tasks = Task.query.order_by('name').filter_by(name='Szukanie')

print(tasks)

In [ ]:
print(type(tasks))

I znów Query. Struktura umożliwiająca łańcuchowanie metod na tyle często pojawia się w programowaniu, że posiada nawet swoją nazwę. Mówimy, że Query implementuje wzorzec fluent interface (płynny interfejs).

Ale nie wszystkie metody Query zwracają Query. Na przykład metody all, first i one zwracają odpowiednio listę wszystkich, pierwszy i jeden element (czyli instancje modelu). Takie metody nie zwracają obiektu klasy Query (nie implementują wzorca fluent interface) i dlatego po ich użyciu nie można już użyć żadnej innej metody obiektu Query do łańcuchowania.

In [ ]:
tasks = Task.query.order_by('name').limit(7).all()
print(tasks)

In [ ]:
task = Task.query.order_by('name').limit(7).first()
print(task)

In [ ]:
task = Task.query.order_by(Task.name.desc()).limit(7).first()
print(task)

Metody klasy Query implementują wzorzec lazy evaluation. Są wykonywane dopiero w momencie konsumowania. Konsumowanie polega na użyciu wartości zwracanych w zapytaniu i jest równoznaczne z wykonaniem zapytania na bazie. Skonsumować obiekt klasy Query można np. poprzez wywołanie jednej z konsumujących metod (all, first, ...), przeiterowanie się po nim czy np. zrzutowaniu obiektu na listę. Jest to moment, w którym wyniki są potrzebne (do wyświetlenia lub np. zapisania do pamięci) i nie można dłużej zwlekać z wykonaniem zapytania na bazie.

In [ ]:
for item in tasks:
    print(item.name)

### Dostęp do wartości w poszczególnych kolumnach wpisu

Do wartości w poszczególnych kolumnach wpisu dostajemy się poprzez notacją obiektową (odwołujemy się do atrybutu instancji modelu). Jaką wartość w kolumnie name ma ostatni wpis z tabelki Task?

In [ ]:
print(task.name)

### Filtry - metody filter, filter_by (klauzula WHERE)

#### Metoda I - filter_by

Metoda ```filter_by``` służy do wykonywania prostych zapytań.

In [ ]:
query = Task.query.filter_by(name="Szukanie")
print(query)
print(type(query))
result = query.all()
print(result)

Parametrem funkcji `filter_by` jest kolumna, po której filtrujemy. Możemy filtrować po kilku kolumnach jednocześnie.

In [ ]:
query = Task.query.filter_by(id=16, name="Szukanie")
print(query)
result = query.all()
print(result)

filter_by zwraca listę instancji modelu spełniających kryterium wyszukiwania. Może być pusta.

In [ ]:
query = Task.query.filter_by(name="Coś czego nie ma w bazie")
print(query)
result = query.all()
print(result)

Może być jednoelementowa

In [ ]:
query = Task.query.filter_by(name="Pranie")
print(query)
result = query.all()
print(result)

#### Metoda II - filter

Metoda ```filter``` Query wykonuje to samo co metoda filter_by (odpowiada klauzuli WHERE). Posiada inne api.

In [ ]:
query = Task.query.filter(Task.name=="Szukanie")
print(query)

Też zwraca obiekt klasy Query.

In [ ]:
print(type(query))

na którym możemy dalej działać.

In [ ]:
result = query.all()
print(result)

ale jako parametr wejściowy przyjmuje całe wyrażenie, dlatego pozwala na formułowaniu bardziej złożonych zapytań. W metodzie filter można używać kilku
operatorów, takich jak:

In [ ]:
# eq
query = Task.query.filter(Task.name=="Malowanie")
print(query.statement)
result = query.all()
print(result)

In [ ]:
# not eq
q = Task.query.filter(Task.name!="Malowanie")
print(q.statement)
result = q.all()
print(result)

In [ ]:
# like
q = Task.query.filter(Task.name.like("%wanie"))
print(q.statement)
result = q.all()
print(result)

In [ ]:
# in
q = Task.query.filter(Task.name.in_(["Malowanie", "Szukanie", "Pływanie"]))
print(q.statement)
result = q.all()
print(result)

In [ ]:
# not in
q = Task.query.filter(~Task.name.in_(["Malowanie", "Szukanie", "Pływanie"]))
print(q.statement)
result = q.all()
print(result)

In [ ]:
# gt, lt, gte, lte
q = Task.query.filter(Task.id > 5)
print(q.statement)
result = q.all()
print(result)

q = Task.query.filter(Task.id <= 4)
print(q.statement)
result = q.all()
print(result)

In [ ]:
# and v1
q = Task.query.filter(Task.name.like("%wanie"), Task.id >=6)
print(q.statement)
result = q.all()
print(result)

In [ ]:
# and v2
from sqlalchemy import and_

q = Task.query.filter(and_(Task.name.like("%wanie"), Task.id >=6))
print(q.statement)
result = q.all()
print(result)

In [ ]:
# and v3
q = Task.query.filter(
    (Task.name.like("%wanie")) & (Task.id >= 6)
)  # najlepiej warunki zamykać w nawiasach
print(q.statement)
result = q.all()
print(result)

In [ ]:
# or v1
from sqlalchemy import or_

q = Task.query.filter(or_(Task.name.like("%wanie"), Task.id >= 6))
print(q.statement)
result = q.all()
print(result)

In [ ]:
# or v2
q = Task.query.filter(
    (Task.name.like("%wanie")) | (Task.id >= 6)                                 
)
print(q.statement)
result = q.all()
print(result)

#### Metoda III - get

W odróżnieniu od metody filter, metoda get zwraca instancję jedną instancje modelu. Jako parametr przyjmuje wartość klucza głównego w tabeli.

In [ ]:
q = Task.query.get(1)  # get - getting task by primary key
print(task)
print(type(task))

Przedawnione, od wersji 2.0 twórcy zalecają używanie metody get sesji, a nie modelu.

In [ ]:
q = db.session.get(Task, 1)
print(q)

Jeżeli w tabeli nie będzie takiego rekordu metoda get zwraca None.

In [ ]:
task = db.session.get(Task, 100)
print(task)

Podsumowując, metoda get w odróżnieniu od metody filter:
* zwraca instancję modelu (a nie obiekty klasy Query)
* jeżeli w wyniku filtrowania otrzymamy pustą odpowiedź zwróci None (a nie pustą listę)

### Indeksowanie, wycinki (operatory LIMIT i OFFSET)

Klasa Query wspiera indeksowanie oraz wycinki

Znajdźmy czwarty wpis w tabeli Task (indeksowanie od 0).

In [ ]:
task = Task.query.all()[3]
print(task)
print(type(task))  # instancja modelu

Znajdźmy pięc pierwszych wpisów w tabeli Task.

In [ ]:
tasks = Task.query.all()[:5]
print(tasks)

Znajdźmy wszystkie wpisy w tabeli Task poza pięcioma pierwszymi.

In [ ]:
tasks = Task.query.all()[5:]
print(tasks)

Znajdźmy co drugi wpis z tabeli Task (wycinki)

In [ ]:
tasks = Task.query.all()[::2]
print(tasks)
print(type(tasks))

**Uwaga!** 

Query w odróżnieniu od listy Pythonowej nie obsługuje negatywnych indeksów.